# Preprocessing 

In [ ]:
import warnings
import time
import os

import dask
from dask.distributed import Client

import numpy as np
import pandas as pd

This notebook will be used to prepare our feature and ground-truth data for our modeling process. At this point, we have a directory of .feather files that contain our features (see this notebook for feature generation). We want to concatentate these together, and perform the necessary operations to achieve a dataframe where a row is one observation unit; its columns are the associated features and the ground truth data that we want to train the model on. Broadly, we want our dataframe to take on the following form:

| observation_unit |  outcome_1 | outcome_2 | feature_1| feature_2 | feature_3
| ----|  ---- | ---- | -- | -- | -- |
| 1   |  y_1 | x_1 | x_2 | x_3 | x_4
| 2   |  y_2 | x_1 | x_2| x_3 | x_4

Where we can run regress `outcome_1` and `outcome_2` on `feature_1`, `feature_2` and `feature_3`

In our case, an observation is one SEA/year. We have ground truth data that summarizes the total crop yield, total crop loss, as well as other pertinent agricultural variables. We want our dataframe described above to look like this:

| SEA | Year | Observed Yield (Tonnes) | Observed Loss (Tonnes) | feature1| feature2 | feature3
| ----| ----| ---- | ---- | -- | -- | -- |
| 1   | 2016 | 72 | 13 | 1.23 | 3.25 | 0.123
| 2   | 2016  | 50 | 7.5 | 0.78| 1.2 | 2.4

In [2]:
# define the file paths for each file
feature_dir = "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/" #insert path to feature .feather files here
file_paths = [
    "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_2013.feather",
    "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_2014.feather",
    "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_2015.feather",
    "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_2016.feather",
    "/capstone/mosaiks/repos/modeling/data/landsat-8_cropmosaiks_features/landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_2017.feather",
]

# read in each file as a dataframe and concatenate them together
df = pd.concat([pd.read_feather(file_path) for file_path in file_paths])

# print the resulting dataframe

NameError: name 'pd' is not defined

### Read in Features

In [ ]:
# Let's read in the new concatenated features:
features = pd.read_feather("/capstone/mosaiks/repos/modeling/data/cropmosaiks_features_landsat8.feather")
features

### Read in Ground-Truth Data

In [ ]:
# Read in the survey data
country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/featurizeme/total.shp')
country_sea

### Spatially join features to Ground-Truth Data

In [ ]:
# Filter country_sea for unique values of 'seq_unq' and 'geometry'
sea_unq_join = country_sea[['sea_unq', 'geometry']].drop_duplicates()

# Display the filtered DataFrame
print(sea_unq_join)


In [ ]:
# Organize the features by growing season
# Carry months October, November, and December over to the following year's data
# These months represent the start of the growing season for the following year's maize yield
year_end = 2022

features['year'] = np.where(
    features['month'].isin([10, 11, 12]),
    features['year'] + 1, 
    features['year'])

features_gs = features[features['year'] <= year_end]

features_gs.sort_values(['year', 'month'], inplace=True)

In [ ]:
# Create a geodataframe of the new features
features_new_gdf = gpd.GeoDataFrame(
    features_gs, 
    geometry = gpd.points_from_xy(x = features_gs.lon, y = features_gs.lat), 
    crs='EPSG:4326'
)

In [ ]:
# Store the 'geometry' column separately before unstacking
geometry_col = features_new_gdf[['lon', 'lat', 'geometry']].drop_duplicates(subset=['lon', 'lat'])

# Perform the unstacking operation without the 'geometry' column
features_gs_no_geometry = features_gs.drop(columns=['geometry'])
features = features_gs_no_geometry.set_index(['lon', 'lat', 'year', 'month']).unstack()
features.columns = features.columns.map(lambda x: '{}_{}'.format(*x))

# Merge the 'geometry' column back into the features DataFrame
features = features.reset_index().merge(geometry_col, on=['lon', 'lat'])


In [ ]:
# Drop the unwanted 'index' and 'geometry' columns
features = features.filter(regex='^(?!index_)')


# Convert the 'features' DataFrame to a GeoDataFrame
features_gdf = gpd.GeoDataFrame(features, geometry=features['geometry'], crs='EPSG:4326')

In [ ]:
# Now lets combine the sea data 
spatial_join = gpd.sjoin(features_gdf, sea_unq_join, how='right', predicate = 'within')

In [ ]:
features_join = spatial_join.merge(country_sea, on=['year', 'sea_unq'], how='inner')
# Drop the redundant independent lon and lat columns because now that they are in a separate geometry column
features_join = features_join.drop(['lon', 'lat', 'geometry_x'], axis = 1)

## Imputation

### Impute missing values

Imputing "manually" by descending group levels imputes NA values in multiple "cascading" steps, decreasing the proportion of inputed values with each step. First, the NA values are imputed at by both `year` and `geometry`, which should yield imputed values that most closely match the feature values that would be present in the data if there was no clouds obscuring the satellite images. Next, the remaining NA values that could not be imputed by both `year` and `district` are imputed by only `district`. Lastly, the remaining NA vlaues that could not be imputed by both `year` and `district` or by just `district` are imputed by `year` only. This option gives the user more control and transparency over how the imputation is executed.

Imputing using `scikit learn`'s simple imputer executes standard imputation, the details of which can be found in the `scikitlearn` documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

The imputation approach depends on the selection made at the top of this notebook for `impute_manual`.

In [ ]:
# Extract the size of the features dataframe
rows, cols = features_join.shape

# compute the number of feature cells in the features dataframe
num_cells = rows * cols
num_cells

In [ ]:
class bcolors:
    BL = '\x1b[1;34m' #GREEN
    GR = '\x1b[1;36m' #GREEN
    YL = '\x1b[1;33m' #YELLOW
    RD = '\x1b[1;31m' #RED
    RESET = '\033[0m' #RESET COLOR

In [ ]:
# Notes: Have to change the year, get an error rn. Also, check to make sure the number of cells is correct
if impute_manual:
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Starting total row count: {bcolors.BL}{ln_ft}{bcolors.RESET}',
          f'\nPre-Impute NaN row count: {bcolors.RD}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPre-Impute NaN row %: {bcolors.RD}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPre-Impute NaN cell %: {bcolors.RD}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 1: Filling NaN values by month, year, and district group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['year', 'sea_unq'], as_index=False) 
                .transform('mean')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by month and district across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['sea_unq'], as_index=False)
                .transform('mean')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.GR}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.GR}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 3: Drop remaining NaN values\n')
    features_join = features_join.dropna(axis=0)
    print(f'Ending total row count: {bcolors.BL}{len(features_join)}{bcolors.RESET}')
else:
    # Store the geometry column separately
    geometry_col = features_join['geometry_y']
    # Remove the geometry column from the DataFrame
    features_join = features_join.drop(columns=['geometry_y'])
    features_join = features_join.set_index(['year', 'sea_unq'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features_join)
    features_join[:] = imputer.transform(features_join)
    features_join = features_join.reset_index()
    # Add the geometry column back to the DataFrame
    features_join['geometry'] = geometry_col